In [1]:
'''
Lucas Atayde, Alp Burke

File responsible for cleaning up our data pipeline. This will prepare our API to query the
data.
'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
usaa = pd.read_csv("./raw/US_Accidents_Dec19.csv", skiprows=range( int(2974335 * .01), 2974335))
# usaa = pd.read_csv("data_prep/raw/US_Accidents_Dec19.csv")


In [27]:
# Step 1: Convert accident ids to integers
usaa["ID"] = usaa["ID"].apply(lambda id: int(id[2:]))

# Drop unnecessary data. Makes computation slower and adds noise
# Country is the same for ALL.
# Source might effect data quality but ehhhh
usaa = usaa.drop(columns=["Source", "Country"])


In [28]:
# Fill in data with normal distribution
# TODO: Make this more sophisticated using nearest neighbors?
t_mean = usaa["Temperature(F)"].mean()
t_std = usaa["Temperature(F)"].std()
usaa["Temperature(F)"] = usaa["Temperature(F)"].apply(lambda val: np.random.normal(t_mean, t_std) if np.isnan(val) else val)

In [52]:
def make_categorical(frame, col):
    frame[col] = frame[col].astype("category").cat.codes

def num_missing(series):
    return series.isnull().sum()

In [53]:
'''
TMC codes about traffic? Not exactly sure, will convert to categorical data
'''
make_categorical(usaa, "TMC")


In [63]:
# End lat and End long are both pretty much empty / worthless
usaa = usaa.drop(columns="End_Lng")
usaa = usaa.drop(columns="End_Lat")

KeyError: "['End_Lng'] not found in axis"

In [65]:
cat = "Description"
print(num_missing(usaa[cat]))
usaa[cat].hist()
usaa[cat].unique()


0
Error in callback <function flush_figures at 0x11b5bf4d0> (for post_execute):


array(['Right lane blocked due to accident on I-70 Eastbound at Exit 41 OH-235 State Route 4.',
       'Accident on Brice Rd at Tussing Rd. Expect delays.',
       'Accident on OH-32 State Route 32 Westbound at Dela Palma Rd. Expect delays.',
       ...,
       'Right hand shoulder blocked due to accident on Castroville Blvd at CA-156.',
       'Right hand shoulder blocked due to accident on US-101 Monterey Rd Northbound before Exit 353 CA-25.',
       'At Highland Ave/Arden Ave - Accident.'], dtype=object)

KeyboardInterrupt: 